In [1]:
import requests
import json
import pandas as pd
import numpy as np
import time
from datetime import datetime, date
import logging
from typing import Tuple
from tqdm import tqdm
import os

In [2]:
# setup logging
logging.basicConfig(filename='price_data.log', format='(%(process)d): %(asctime)s - %(levelname)s - %(message)s')

# create output directory
outdir = '../data/price_histories/'
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [4]:
# set constant values
FROM_DATE = date(2017, 1, 1)
TO_DATE = date(2021, 12, 31)

DAYS_LIMIT = (TO_DATE - FROM_DATE).days
TO_TS = time.mktime(TO_DATE.timetuple())

UNIT = 'USD'

# fetch historic price data
base_url = 'https://min-api.cryptocompare.com/data/v2'
headers = {
    'Accepts': 'application/json',
    'Authorization': 'Apikey 2572b1c288c45f00c57f9a0dfa1a0da93afc454365cfaaf7ee17820f898fa7b7',
}

def get_price_data(symbol: str) -> Tuple[pd.DataFrame, bool]:
    success = True
    url = base_url + f'/histoday?fsym={symbol}&tsym={UNIT}&toTs={TO_TS}&limit={DAYS_LIMIT}'
    try:
        res = requests.get(url, headers=headers)
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    data = json.loads(res.text)
    # check success
    if data['Response'] != 'Success' or data['Message'] != '':
        msg = f'Unusual response for {symbol}: [{data["Response"]}: {data["Message"]}]'
        logging.warning(msg)
    if data['HasWarning'] == True:
        print(data)
        input('Unexpected Warning, press any key to continue')
    if not 'Data' in data or not 'Data' in data['Data']:
        success = False
        return None, success
    else:
        df = pd.DataFrame.from_dict(data['Data']['Data'])
        return df, success


### 1st Pull: *coin.json*

In [ ]:
# read coin list
with open('../data/coin.json', 'r') as f:
    coins = json.load(f)
df_coins = pd.DataFrame(list(coins.items()), columns=['symbol', 'name'])

# get price history
start = 0
symbols_skipped = []

for index, row in tqdm(df_coins.iloc[start:].iterrows()):
    symbol = row['symbol']
    df, success = get_price_data(symbol)
    if success == True:
        df.to_csv(outdir + f'{symbol}.csv')
    else:
        symbols_skipped.append(symbol)

# write skipped symbols to csv file
df_symbols_skipped = pd.DataFrame(symbols_skipped)
df_symbols_skipped.to_csv(outdir + 'symbols_skipped_price.csv', mode='a', header=False, index=False)

### 2nd Pull: cryptoCompare full archive (remainder)

In [5]:
# read coin list which has already been pulled
with open('../data/coin.json', 'r') as f:
    ex_coins = json.load(f)
ex_coins_sym = list(ex_coins.keys())

# read new coin archive
with open('../data/coin_info.json', 'r') as f:
    coins = json.load(f)
coins_sym = list(coins.keys())

# remove previous symbols
new_coins_sym = [sym for sym in coins_sym if sym not in ex_coins_sym]
len(new_coins_sym)

4123

In [7]:
# get price history
start = 0
symbols_skipped = []

for symbol in tqdm(new_coins_sym[start:]):
    df, success = get_price_data(symbol)
    if success == True:
        df.to_csv(outdir + f'{symbol}.csv')
    else:
        symbols_skipped.append(symbol)

# append skipped symbols to csv file
df_symbols_skipped = pd.DataFrame(symbols_skipped)
df_symbols_skipped.to_csv(outdir + 'symbols_skipped_price.csv', mode='a', header=False, index=False)

100%|██████████| 4122/4122 [24:37<00:00,  2.79it/s]  
